<a href="https://colab.research.google.com/github/chw8207/Transformer/blob/main/%EA%B8%B0%EA%B3%84%EB%B2%88%EC%97%AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip tensorflow-text==2.8.0

ERROR: unknown command "tensorflow-text==2.8.0"


In [ ]:
!pip install tensorflow-text

In [38]:
import tensorflow as tf
import pickle
from pickle import dump
import re
import string
import unicodedata
from pickle import load
from pickle import dump
from collections import Counter
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
import os
import numpy as np
# import tensorflow_text

### GPU 설정

In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0' :
    raise SystemError('GPU device not found')
print(f'Found GPU at: {device_name}')

Found GPU at: /device:GPU:0


In [3]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14374675455204084798
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 40129593344
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12714923305354788496
physical_device_desc: "device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:00:04.0, compute capability: 8.0"
xla_global_id: 416903419
]


### WMT 데이터셋 전처리

#### 원시 데이터 전처리

In [4]:
# 파일을 메모리에 로드하는 함수 정의
def load_doc(filename) :
  # 읽기 전용으로 파일 열기
  file = open(filename, mode='rt', encoding='utf-8')
  # 모든 텍스트 읽기
  text = file.read()
  # 파일 닫기
  file.close()
  return text

# 로드된 문서는 문장들로 분할됨
def to_sentences(doc) :
  return doc.strip().split('\n')

# 가장 짧은/가장 긴 문장의 길이 찾기
def sentence_lengths(sentences) :
  lengths = [len(s.split()) for s in sentences]
  return min(lengths), max(lengths)

In [5]:
# 불러온 문장 라인 정리(노이즈 제거)
def clean_lines(lines) :
  cleaned = list()
  # char filtering을 위해 regex 준비하기
  re_print = re.compile('[^%s]' % re.escape(string.printable))
  # 구두점 제거하기 위한 번역표 준비하기
  table = str.maketrans('','',string.punctuation)
  for line in lines :
    # 유니코드 characters 정규화
    # 비 아스키 문자 제거
    # NFD : 엑센트와 같은 결합 마디를 기본 형태의 문자로 분리함.
    line = unicodedata.normalize('NFD',line).encode('ascii', 'ignore')
    line = line.decode('UTF-8')
    # 공백에서 토큰화
    line = line.split()
    # 소문자로 변환
    line = [word.lower() for word in line]
    # 각 토큰에서 구두점 제거
    line = [word.translate(table) for word in line]
    # 각 토큰에서 프린트할 수 없는 문자 제거
    # 문자 w에서 프린트할 수 없는 문자를 빈문자로 대체함.
    line = [re_print.sub('',w) for w in line]
    # 숫자가 있는 토큰 제거
    line = [word for word in line if word.isalpha]
    # 문자열 형태로 저장(공백문자를 사용하여 하나로 연결)
    cleaned.append(' '.join(line))

  return cleaned

In [6]:
# 영어 데이터 로드하기
filename = '/content/drive/MyDrive/Colab Notebooks/Transformer/data/Europarl.en-fr.en'
doc = load_doc(filename)
sentences = to_sentences(doc)
minlen, maxlen = sentence_lengths(sentences)
print('English data: sentences=%d, min=%d, max=%d' % (len(sentences), minlen, maxlen))
cleanf = clean_lines(sentences)

English data: sentences=2051014, min=1, max=668


In [7]:
# pickle이 English.pkl이라는 직렬화된 파일에 덤프한다.
filename = 'English.pkl'
outfile = open(filename,'wb')
pickle.dump(cleanf, outfile)
outfile.close()
print(filename, ' saved')

English.pkl  saved


In [8]:
# 프랑스어 데이터 로드하기
filename = '/content/drive/MyDrive/Colab Notebooks/Transformer/data/Europarl.en-fr.fr'
doc = load_doc(filename)
sentences = to_sentences(doc)
minlen, maxlen = sentence_lengths(sentences)
print('English data: sentences=%d, min=%d, max=%d' % (len(sentences), minlen, maxlen))
cleanf = clean_lines(sentences)

English data: sentences=2051014, min=1, max=693


In [9]:
# pickle이 English.pkl이라는 직렬화된 파일에 덤프한다.
filename = 'French.pkl'
outfile = open(filename,'wb')
pickle.dump(cleanf, outfile)
outfile.close()
print(filename, ' saved')

French.pkl  saved


#### 데이터셋 전처리 마무리
- 섹션에서 정리된 데이터셋을 로드한 후 마무리된 전처리를 저장하는 함수 정의하기

In [10]:
# clean 데이터셋 로드히기
def load_clean_sentences(filename) :
  return load(open(filename, 'rb'))

# clean 문장의 리스트를 파일로 저장하기
def save_clean_sentences(sentences, filename) :
  dump(sentences, open(filename, 'wb'))
  print('Saved: %s' % filename)

# 모든 단어에 대한 빈도표 생성하기
def to_vocab(lines) :
  vocab = Counter()
  for line in lines :
    tokens = line.split()
    vocab.update(tokens)
  return vocab

# 어휘 카운터는 빈도가 min_occurance 미만인 단어를 검출함
def trim_vocb(vocab, min_occurance) :
  tokens = [k for k,c in vocab.items() if c >= min_occurance]
  return set(tokens)

In [11]:
# OOV 단어를 unk(알 수 없는) 토큰으로 대체
def update_dataset(lines, vocab) :
  new_lines = list()
  for line in lines :
    new_tokens = list()
    for token in line.split() :
      if token in vocab :
        new_tokens.append(token)
      else :
        new_tokens.append('unk')
    new_line = ' '.join(new_tokens)
    new_lines.append(new_line)
  return new_lines

In [12]:
# 영어 데이터셋
filename = 'English.pkl'
lines = load_clean_sentences(filename)
# 어휘 계산
vocab = to_vocab(lines)
print('English Vocabulary: %d' % len(vocab))
# 어휘 감소
vocab = trim_vocb(vocab, 5)
print('New English Vocabulary: %d' % len(vocab))
# mark out of vocabulary words
lines = update_dataset(lines, vocab)
# 수정된 데이터셋 저장하기
filename = 'english_vocab.pkl'
save_clean_sentences(lines, filename)
# spot check
for i in range(20) :
  print('line',i,':',lines[i])

English Vocabulary: 133407
New English Vocabulary: 44574
Saved: english_vocab.pkl
line 0 : resumption of the session
line 1 : i declare resumed the session of the european parliament adjourned on friday 17 december 1999 and i would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period
line 2 : although as you will have seen the dreaded millennium bug failed to materialise still the people in a number of countries suffered a series of natural disasters that truly were dreadful
line 3 : you have requested a debate on this subject in the course of the next few days during this partsession
line 4 : in the meantime i should like to observe a minute s silence as a number of members have requested on behalf of all the victims concerned particularly those of the terrible storms in the various countries of the european union
line 5 : please rise then for this minute s silence
line 6 : the house rose and observed a minute s silence
line 7 : madam pre

In [13]:
# 프랑스어 데이터셋
filename = 'French.pkl'
lines = load_clean_sentences(filename)
# 어휘 계산
vocab = to_vocab(lines)
print('French Vocabulary: %d' % len(vocab))
# 어휘 감소
vocab = trim_vocb(vocab, 5)
print('New French Vocabulary: %d' % len(vocab))
# mark out of vocabulary words
lines = update_dataset(lines, vocab)
# 수정된 데이터셋 저장하기
filename = 'french_vocab.pkl'
save_clean_sentences(lines, filename)
# spot check
for i in range(20) :
  print('line',i,':',lines[i])

French Vocabulary: 171009
New French Vocabulary: 61764
Saved: french_vocab.pkl
line 0 : reprise de la session
line 1 : je declare reprise la session du parlement europeen qui avait ete interrompue le vendredi 17 decembre dernier et je vous renouvelle tous mes vux en esperant que vous avez passe de bonnes vacances
line 2 : comme vous avez pu le constater le grand bogue de lan 2000 ne sest pas produit en revanche les citoyens dun certain nombre de nos pays ont ete victimes de catastrophes naturelles qui ont vraiment ete terribles
line 3 : vous avez souhaite un debat a ce sujet dans les prochains jours au cours de cette periode de session
line 4 : en attendant je souhaiterais comme un certain nombre de collegues me lont demande que nous observions une minute de silence pour toutes les victimes des tempetes notamment dans les differents pays de lunion europeenne qui ont ete touches
line 5 : je vous invite a vous lever pour cette minute de silence
line 6 : le parlement debout observe une mi

### BLUE에 의한 기계 번역 평가

#### 기하적 평가

In [14]:
# 예시 1
reference = [['the', 'cat', 'likes', 'milk'], ['cat', 'likes', 'milk']]
candidate = ['the', 'cat', 'likes', 'milk']
score = sentence_bleu(reference, candidate)
print('Emxample 1', score)

Emxample 1 1.0


In [15]:
# 예시 2
reference = [['the', 'cat', 'likes', 'milk']]
candidate = ['the', 'cat', 'likes', 'milk']
score = sentence_bleu(reference, candidate)
print('Emxample 2', score)

Emxample 2 1.0


In [16]:
# 3-gram 오버랩을 찾고 있는 경우
reference = [['the', 'cat', 'likes', 'milk']]
candidate = ['the', 'cat', 'enjoys', 'milk']
score = sentence_bleu(reference, candidate)
print('Emxample 3', score)

Emxample 3 1.0547686614863434e-154


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


#### 스무딩 기법 적용 - chencherry smoothing

In [17]:
# 예제 4(불어-영어 예제)
reference = [['je','vous','invite', 'a', 'vous', 'lever','pour', 'cette', 'minute', 'de', 'silence']]
candidate = ['levez','vous','svp','pour', 'cette', 'minute', 'de', 'silence']
score = sentence_bleu(reference, candidate)
print('without smoothing score', score)

without smoothing score 0.37188004246466494


In [18]:
# 평가에 개방적 스무딩을 추가하기
chencherry = SmoothingFunction()
r1 = list('je vous invite a vous lever pour cette minute de silence')
candidate = list('levez vous svp pour cette minute de silence')
print('with smoothing score', sentence_bleu([r1], candidate, smoothing_function=chencherry.method1))

with smoothing score 0.6194291765462159


### Trax에 의한 번역

In [19]:
# Trax 설치하기
!pip install -q -U trax

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 637.9/637.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 91.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 107.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 38.9 MB/s eta 0:00:00
